### Classes and Methods


In [11]:
# dev comment: DNAstring() and ParseFasta() lack encapsulation, should probably be rewritten...

In [8]:
class DNAstring:
    def __init__(self, s):
        self.s = s
    def fwdStrand(self):
        return self.s
    def revStrand(self):
        revDNA = self.s
        return revDNA[::-1]
    def compStrand(self):
        compDNA = self.s
        compDNA_string = ''
        for i in compDNA:
            if i == 'A':
                compDNA_string += 'T'
            elif i == 'T':
                compDNA_string += 'A'
            elif i == 'C':
                compDNA_string += 'G'
            elif i == 'G':
                compDNA_string += 'C'
            else:
                print('Error, unfamiliar base')
        return compDNA_string
    def revCompStrand(self):
        s_rev = self.s[::-1]
        s_rc = ''
        for i in s_rev:
            if i == 'A':
                s_rc += 'T'
            elif i == 'T':
                s_rc += 'A'
            elif i == 'C':
                s_rc += 'G'
            elif i == 'G':
                s_rc += 'C'
            else:
                print('Error, unfamiliar base')
        return s_rc
class ParseFasta:
    def __init__(self, f):
        self.f = f
    def parseFastaFile(self):
        fastaFile = self.f
        l = []
        with open(fastaFile, 'r') as fOpen:
            h = ''
            for i in fOpen:
                i = i.rstrip('\r\n')
                if i == '':
                    continue
                if i[0] == '>':
                    h = i
                else:
                    l.append((h,i))
        return l
    def parseFastaFileWithRC(self):
        fastaFile = self.f
        l = []
        with open(fastaFile, 'r') as fOpen:
            h = ''
            for i in fOpen:
                i = i.rstrip('\r\n')
                if i == '':
                    continue
                if i[0] == '>':
                    h = i
                else:
                    h_f = h + '_f'
                    h_rc = h + '_rc'
                    l.append((h_f,i))
                    i_rc = DNAstring(i).revCompStrand()
                    l.append((h_rc, i_rc))
        return l
    def parseFastaOutputFile(self):
        fastaFile = self.f
        l = []
        with open(fastaFile, 'r') as fOpen:
            for i in fOpen:
                i = i.rstrip('\r\n')
                l.append(i)
        return l
class OutputFasta:
    def __init__(self):
        self.l = []
    def createOutputList(self, c):
        self.l = c
    def outputFastaToFile(self, fName):
        if not self.l:
            print('Error, call createOutputList() function first!')
            return None
        for i in self.l:
            with open(fName, 'a') as fWrite:
                fWrite.write(i[0] + '\n')
                fWrite.write(i[1] + '\n')
class ParseOutputLine:
    def __init__(self):
        self.l = []
    def get_parseLine(self):
        return self.l
    def set_parseLine(self, b, s):
        if b:
            parsedLineForExon = []
            parse1 = s[2:] # remove '# ' from line
            leadingCharsRemoved = parse1.split(',')
            leadingCharRemovedFromAll = [x[1:] for x in leadingCharsRemoved] # list is now ['seq1:exon1', 'seq2:exon2',...]
            parsedLineAsLoL = [x.split(':')[:-1] for x in leadingCharRemovedFromAll]
            self.l = [':'.join(x) for x in parsedLineAsLoL] # output is ['seq1:01:01', 'seq2:01',...]
        else:
            parse1 = s[1:]
            splitParse = parse1.split(' ')
            self.l = parse1.split(' ')
class ParseOutputFile:
    def __init__(self):
        self.o = []
    def get_parseFile(self):
        return self.o
    def set_parseFile(self, f):
        returnedList = []
        parsedAlleleLine = ParseOutputLine()
        parsedFqHeaderLine = ParseOutputLine()
        if not f:
            return None
        for i in f:
            if i[0] == '#':
                parsedAlleleLine.set_parseLine(True, i)
            elif i[0] == '>':
                parsedFqHeaderLine.set_parseLine(False, i)
            else:
                returnedList.append((parsedAlleleLine, parsedFqHeaderLine, [i]))
                parsedFqHeaderLine = ParseOutputLine()
                parsedAlleleLine = ParseOutputLine()
        # return returnedList
        self.o = returnedList

In [ ]:
%%bash

# comments:
# skip this cell, included as a reference only
# depending on the workflow, you can either use a single sample or concatenate multiple samples
# but you MUST keep read pairs separate
cat 39526_Mamu-exon*_matches_r1* > 39526_Mamu-exonAll_allR1.txt
cat 39526_Mamu-exon*_matches_r2* > 39526_Mamu-exonAll_allR2.txt

In [30]:
def quickCheck(d):
    ct = 0
    if type(d) is dict:
        for k,v in d.items():
            ct += 1
            if ct < 10:
                print(k,v)
            else:
                break
    else:
        try:
            for i in d:
                ct += 1
                if ct < 10:
                    print(i)
                else:
                    break
        except TypeError:
            print('Not a list or set, cannot show preview')

In [55]:
parsedFile = ParseFasta('39526_Mamu-exonAll_allR1.txt').parseFastaOutputFile()
parsedFileObject_R1 = ParseOutputFile()
parsedFileObject_R1.set_parseFile(parsedFile)
parsedFileAsList_R1 = parsedFileObject_R1.get_parseFile()
parsedDictOfReads_R1 = dict()
for i in parsedFileAsList_R1:
    k = i[1].get_parseLine()[0]
    v = i[0].get_parseLine()
    if k not in parsedDictOfReads_R1:
        parsedDictOfReads_R1[k] = v
    else:
        tmp = parsedDictOfReads_R1[k]
        tmp.extend(v)
        tmpSet = set(tmp)
        parsedDictOfReads_R1[k] = list(tmpSet)
print('R1\n')
quickCheck(parsedDictOfReads_R1)

parsedFile = ParseFasta('39526_Mamu-exonAll_allR2.txt').parseFastaOutputFile()
parsedFileObject_R2 = ParseOutputFile()
parsedFileObject_R2.set_parseFile(parsedFile)
parsedFileAsList_R2 = parsedFileObject_R2.get_parseFile()
parsedDictOfReads_R2 = dict()
for i in parsedFileAsList_R2:
    k = i[1].get_parseLine()[0]
    v = []
    v = i[0].get_parseLine()
    if k not in parsedDictOfReads_R2:
        parsedDictOfReads_R2[k] = v
    else:
        tmp = parsedDictOfReads_R2[k]
        tmp.extend(v)
        tmpSet = set(tmp)
        parsedDictOfReads_R2[k] = list(tmpSet)
print('\nR2\n')
quickCheck(parsedDictOfReads_R2)

R1

A00433:14:H3W3NDSXX:1:1112:17951:18427 ['Mamu-A1*008:01:01', 'Mamu-A1*027:01', 'Mamu-A1*008:01:04', 'Mamu-A1*006:03', 'Mamu-A1*090:01', 'Mamu-A1*008:03', 'Mamu-A1*003:12', 'Mamu-A1*008:02', 'Mamu-A1*006:01', 'Mamu-A1*006:02', 'Mamu-A1*003:13', 'Mamu-A1*003:04', 'Mamu-A1*065:02']
A00433:14:H3W3NDSXX:1:1126:18945:3458 ['Mamu-B*007:03', 'Mamu-B*139:01', 'Mamu-B*007:07', 'Mamu-B*070:06', 'Mamu-B*070:02', 'Mamu-B*007:06', 'Mamu-B*078:01', 'Mamu-A2*01:01', 'Mamu-B*007:04:01', 'Mamu-B*070:04', 'Mamu-B*007:02:01', 'Mamu-B*007:01', 'Mamu-B*070:07', 'Mamu-B*070:01', 'Mamu-B*106:02', 'Mamu-B*070:05']
A00433:14:H3W3NDSXX:1:1158:25075:16485 ['Mamu-B*092:01', 'Mamu-B*044:01:02', 'Mamu-B*027:03', 'Mamu-B*044:05', 'Mamu-B*003:02', 'Mamu-B*044:01:01', 'Mamu-B*044:03', 'Mamu-B*050:02', 'Mamu-B*061:02', 'Mamu-B*064:01', 'Mamu-I*01:09', 'Mamu-B*038:02', 'Mamu-I*01:09:02', 'Mamu-B*092:02', 'Mamu-B*145:01', 'Mamu-B*027:04', 'Mamu-B*015:04:02', 'Mamu-B*034:04', 'Mamu-B*187:02', 'Mamu-B*039:02', 'Mamu-B*0

In [58]:
listOfR1_keys = parsedDictOfReads_R1.keys()
listOfR2_keys = parsedDictOfReads_R2.keys()
listOfR1R2_keys = []
for k in listOfR1_keys:
    if k in listOfR2_keys:
        listOfR1R2_keys.append(k)
for k in listOfR1R2_keys:
    check_R1List = parsedDictOfReads_R1[k]
    check_R2List = parsedDictOfReads_R2[k]
    check_R1List.sort()
    check_R2List.sort()
    if check_R1List == check_R2List:
        print(str(k) + ' matches to')
        print(parsedDictOfReads_R1[k])
    else:
        print('Warning!  Check:')
        print(check_R1List)
        print(check_R2List)

Warning!  Check:
['Mamu-DRB*W20:04', 'Mamu-DRB*W27:03', 'Mamu-DRB*W27:07', 'Mamu-DRB*W28:02', 'Mamu-DRB*W3:15:01', 'Mamu-DRB*W40:03', 'Mamu-DRB*W4:02:02', 'Mamu-DRB*W4:07', 'Mamu-DRB*W4:08', 'Mamu-DRB*W59:02', 'Mamu-DRB*W7:02:02', 'Mamu-DRB*W7:03', 'Mamu-DRB*W7:05', 'Mamu-DRB1*10:16', 'Mamu-DRB3*04:09']
['Mamu-DRB*W2:02', 'Mamu-DRB1*04:12']
Warning!  Check:
['Mamu-DRB*W20:04', 'Mamu-DRB*W27:03', 'Mamu-DRB*W27:07', 'Mamu-DRB*W28:02', 'Mamu-DRB*W3:15:01', 'Mamu-DRB*W40:03', 'Mamu-DRB*W4:02:02', 'Mamu-DRB*W4:07', 'Mamu-DRB*W4:08', 'Mamu-DRB*W59:02', 'Mamu-DRB*W7:02:02', 'Mamu-DRB*W7:03', 'Mamu-DRB*W7:05', 'Mamu-DRB1*10:16', 'Mamu-DRB3*04:09']
['Mamu-DRB*W2:02', 'Mamu-DRB1*04:12']
Warning!  Check:
['Mamu-DQB1*17:06:01', 'Mamu-DQB1*17:07', 'Mamu-DQB1*17:14', 'Mamu-DQB1*18:02', 'Mamu-DQB1*18:10', 'Mamu-DQB1*18:12', 'Mamu-DQB1*18:19:01', 'Mamu-DQB1*18:19:02', 'Mamu-DQB1*18:20', 'Mamu-DQB1*18:22', 'Mamu-DQB1*18:24', 'Mamu-DQB1*18:27']
['Mamu-DQB1*06:02:01', 'Mamu-DQB1*06:09', 'Mamu-DQB1*06:14'

In [47]:
if 'A00433:14:H3W3NDSXX:1:1112:17951:18427' in parsedDictOfReads_R2:
    print('found key in R2 dict')
if 'A00433:14:H3W3NDSXX:1:1112:17951:18427' in parsedDictOfReads_R1:
    print('found key in R1 dict')

found key in R2 dict
found key in R1 dict


In [37]:
listOfKeys_R1 = 

v1 = ['Mamu-A1*008:01:01', 'Mamu-A1*027:01', 'Mamu-A1*008:01:04', 'Mamu-A1*006:03', 'Mamu-A1*090:01', 'Mamu-A1*008:03', 'Mamu-A1*003:12', 'Mamu-A1*008:02', 'Mamu-A1*006:01', 'Mamu-A1*006:02', 'Mamu-A1*003:13', 'Mamu-A1*003:04', 'Mamu-A1*065:02']
v2 = ['Mamu-A1*008:01:01', 'Mamu-A1*027:01', 'Mamu-A1*008:01:04', 'Mamu-A1*006:03', 'Mamu-A1*090:01', 'Mamu-A1*008:03', 'Mamu-A1*003:12', 'Mamu-A1*008:02', 'Mamu-A1*006:01', 'Mamu-A1*006:02', 'Mamu-A1*003:13', 'Mamu-A1*003:04', 'Mamu-A1*065:02']
v1.sort()
v2.sort()
if v1 == v2:
    print('same')

same


In [ ]:
ct = 0
for i in parsedFileAsList:
    ct += 1
    if ct < 10:
        print(i[0].get_parseLine())
        print(i[1].get_parseLine())
        print(i[2])
    else:
        break

